# Validate Pre-Season Testing Predictions

**Goal:** Test if pre-season testing predicts race outcomes.

**Methodology:**
1. Extract testing for years 2022-2025
2. Get car directionality (relative strengths/weaknesses)
3. Match to track characteristics
4. Predict first 5 races (early season)
5. Compare to actual results

**Key question:** Is 2022 (regulation change) more predictable than stable years?

In [6]:
import json
import warnings
from pathlib import Path

import fastf1
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import spearmanr

warnings.filterwarnings('ignore')

import logging

logging.getLogger("fastf1").setLevel(logging.ERROR)

# Enable cache
cache_dir = Path("../data/raw/.fastf1_cache")
cache_dir.mkdir(parents=True, exist_ok=True)
fastf1.Cache.enable_cache(str(cache_dir))

print("✓ Setup complete")

✓ Setup complete


## Step 1: Load Track Profiles (from cache)

In [7]:
track_profiles_path = Path("../data/processed/track_characteristics/track_profiles_cache.json")

if track_profiles_path.exists():
    with open(track_profiles_path) as f:
        track_profiles = json.load(f)
    print(f"✓ Loaded {len(track_profiles)} track profiles from cache")
else:
    print("ERROR: Track profiles not found!")
    print("Run testing_to_season_prediction.ipynb first to generate cache.")
    track_profiles = {}

✓ Loaded 23 track profiles from cache


## Step 2: Extract Testing Directionality

In [8]:
def extract_testing_directionality(year):
    """Extract car directionality from pre-season testing for ONE year."""
    print(f"\nExtracting {year} pre-season testing...")
    
    try:
        schedule = fastf1.get_event_schedule(year)
        testing_events = schedule[schedule['EventFormat'] == 'testing']
        
        if len(testing_events) == 0:
            print("  No testing data")
            return None
        
        # Collect all testing data
        team_data = {}
        
        for idx, event in testing_events.iterrows():
            event_name = event['EventName']
            
            for day in [1, 2, 3]:
                try:
                    session = fastf1.get_session(year, event_name, day)
                    session.load(laps=True, telemetry=True)
                    
                    laps = session.laps[
                        (session.laps['IsAccurate'] == True) & 
                        (session.laps['TrackStatus'] == '1')
                    ]
                    
                    for team in laps['Team'].unique():
                        team_laps = laps[laps['Team'] == team]
                        if len(team_laps) < 5:
                            continue
                        
                        try:
                            fastest = team_laps.pick_fastest()
                            tel = fastest.get_telemetry()
                            
                            if tel is None or len(tel) == 0:
                                continue
                            
                            if team not in team_data:
                                team_data[team] = []
                            
                            team_data[team].append({
                                'max_speed': tel['Speed'].max(),
                                'slow_corner_speed': tel[(tel['Speed'] >= 0) & (tel['Speed'] < 100)]['Speed'].mean(),
                                'medium_corner_speed': tel[(tel['Speed'] >= 100) & (tel['Speed'] < 200)]['Speed'].mean(),
                                'high_corner_speed': tel[(tel['Speed'] >= 200) & (tel['Speed'] < 250)]['Speed'].mean()
                            })
                        except:
                            pass
                except:
                    pass
        
        # Average across testing
        team_avg = {}
        for team, metrics_list in team_data.items():
            df = pd.DataFrame(metrics_list)
            team_avg[team] = df.mean().to_dict()
        
        if len(team_avg) == 0:
            print("  No valid data")
            return None
        
        # Calculate directionality (relative to average)
        metrics = ['max_speed', 'slow_corner_speed', 'medium_corner_speed', 'high_corner_speed']
        
        # Get averages
        avg_values = {}
        for metric in metrics:
            values = [team_avg[team][metric] for team in team_avg 
                     if not np.isnan(team_avg[team][metric])]
            if len(values) > 0:
                avg_values[metric] = np.mean(values)
            else:
                avg_values[metric] = 0
        
        # Calculate relative performance
        directionality = {}
        for team, chars in team_avg.items():
            directionality[team] = {}
            for metric in metrics:
                value = chars[metric]
                if not np.isnan(value) and avg_values[metric] > 0:
                    directionality[team][metric] = (value - avg_values[metric]) / avg_values[metric]
                else:
                    directionality[team][metric] = 0.0
        
        print(f"  ✓ {len(directionality)} teams")
        return directionality
        
    except Exception as e:
        print(f"  Error: {e}")
        return None

# Extract for 2022-2025
print("="*60)
print("Extracting testing data...")
print("="*60)

testing_data = {}
for year in [2022, 2023, 2024, 2025]:
    result = extract_testing_directionality(year)
    if result:
        testing_data[year] = result

print(f"\n✓ Extracted {len(testing_data)} years")
if 2022 in testing_data:
    print("  → 2022 included (NEW REGULATIONS - regulation change)")

Extracting testing data...

Extracting 2022 pre-season testing...
  ✓ 10 teams

Extracting 2023 pre-season testing...
  ✓ 10 teams

Extracting 2024 pre-season testing...
  ✓ 10 teams

Extracting 2025 pre-season testing...
  ✓ 10 teams

✓ Extracted 4 years
  → 2022 included (NEW REGULATIONS - regulation change)


## Step 3: Helper Functions

In [9]:
def calculate_track_suitability(car_directionality, track_profile):
    """Calculate how well a car suits a track."""
    total_pct = (
        track_profile['straights_pct'] +
        track_profile['slow_corners_pct'] +
        track_profile['medium_corners_pct'] +
        track_profile['high_corners_pct']
    )
    
    if total_pct == 0:
        return 0.0
    
    modifier = (
        car_directionality.get('max_speed', 0) * (track_profile['straights_pct'] / total_pct) +
        car_directionality.get('slow_corner_speed', 0) * (track_profile['slow_corners_pct'] / total_pct) +
        car_directionality.get('medium_corner_speed', 0) * (track_profile['medium_corners_pct'] / total_pct) +
        car_directionality.get('high_corner_speed', 0) * (track_profile['high_corners_pct'] / total_pct)
    )
    
    return modifier

def get_race_results(year, race_name):
    """Get actual race results."""
    try:
        session = fastf1.get_session(year, race_name, 'R')
        session.load(laps=False)
        
        results = session.results[['TeamName', 'Position', 'Points']].copy()
        results = results[results['Position'].notna()]
        
        # Best position per team
        team_results = results.groupby('TeamName').agg({
            'Position': 'min',
            'Points': 'sum'
        }).sort_values('Position')
        
        return team_results
    except:
        return None

def predict_race_order(directionality, race_name, track_profiles):
    """Predict race order using testing directionality."""
    if race_name not in track_profiles:
        return None
    
    track = track_profiles[race_name]
    scores = {}
    
    for team, car in directionality.items():
        scores[team] = calculate_track_suitability(car, track)
    
    predicted = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return predicted

def calculate_accuracy(predicted, actual_results):
    """Calculate Spearman correlation between predicted and actual."""
    common_teams = set([t[0] for t in predicted]) & set(actual_results.index)
    
    if len(common_teams) < 3:
        return None
    
    pred_ranks = {team: i for i, (team, _) in enumerate(predicted, 1)}
    actual_ranks = {team: int(actual_results.loc[team, 'Position']) 
                   for team in common_teams}
    
    pred_list = [pred_ranks[t] for t in common_teams]
    actual_list = [actual_ranks[t] for t in common_teams]
    
    corr, _ = spearmanr(pred_list, actual_list)
    return corr

print("✓ Helper functions defined")

✓ Helper functions defined


## Step 4: Validate Predictions

In [10]:
print("="*60)
print("VALIDATING PREDICTIONS")
print("="*60)

results = []

for year in sorted(testing_data.keys()):
    is_reg_change = (year == 2022)
    
    print(f"\n{year}" + (" [NEW REGULATIONS]" if is_reg_change else " [Stable]"))
    print("-" * 60)
    
    try:
        schedule = fastf1.get_event_schedule(year)
        races = schedule[schedule['EventFormat'] == 'conventional'].head(5)
        
        for idx, race in races.iterrows():
            race_name = race['EventName']
            
            # Get actual results
            actual = get_race_results(year, race_name)
            if actual is None:
                print(f"  {race_name[:30]:30s} - No data")
                continue
            
            # Predict
            predicted = predict_race_order(testing_data[year], race_name, track_profiles)
            if predicted is None:
                print(f"  {race_name[:30]:30s} - No track profile")
                continue
            
            # Calculate accuracy
            corr = calculate_accuracy(predicted, actual)
            if corr is None:
                print(f"  {race_name[:30]:30s} - Cannot calculate")
                continue
            
            print(f"  {race_name[:30]:30s} Correlation: {corr:.3f}")
            
            results.append({
                'year': year,
                'race': race_name,
                'correlation': corr,
                'regulation_change': is_reg_change
            })
    
    except Exception as e:
        print(f"  Error: {e}")

print(f"\n✓ Validated {len(results)} races")

VALIDATING PREDICTIONS

2022 [NEW REGULATIONS]
------------------------------------------------------------
  Bahrain Grand Prix             Correlation: -0.370
  Saudi Arabian Grand Prix       Correlation: 0.406
  Australian Grand Prix          Correlation: 0.406
  Miami Grand Prix               Correlation: 0.261
  Spanish Grand Prix             Correlation: -0.018

2023 [Stable]
------------------------------------------------------------
  Bahrain Grand Prix             Correlation: 0.248
  Saudi Arabian Grand Prix       Correlation: 0.200
  Australian Grand Prix          Correlation: -0.042
  Miami Grand Prix               Correlation: 0.370
  Monaco Grand Prix              Correlation: 0.552

2024 [Stable]
------------------------------------------------------------
  Bahrain Grand Prix             Correlation: 0.830
  Saudi Arabian Grand Prix       Correlation: 0.612
  Australian Grand Prix          Correlation: 0.042
  Japanese Grand Prix            Correlation: 0.745
  Emilia 

## Step 5: Analysis

In [11]:
if len(results) == 0:
    print("No results to analyze")
else:
    df = pd.DataFrame(results)
    
    print("\n" + "="*60)
    print("SUMMARY")
    print("="*60)
    
    # Overall
    print(f"\nOverall average correlation: {df['correlation'].mean():.3f}")
    
    # By year
    print("\nBy year:")
    for year in sorted(df['year'].unique()):
        year_data = df[df['year'] == year]
        avg = year_data['correlation'].mean()
        label = " [NEW REGS]" if year == 2022 else " [Stable]"
        print(f"  {year}{label:12s} {avg:.3f}")
    
    # Regulation change comparison
    reg_change = df[df['regulation_change'] == True]
    stable = df[df['regulation_change'] == False]
    
    if len(reg_change) > 0 and len(stable) > 0:
        print("\n" + "="*60)
        print("REGULATION CHANGE COMPARISON")
        print("="*60)
        
        reg_avg = reg_change['correlation'].mean()
        stable_avg = stable['correlation'].mean()
        diff = reg_avg - stable_avg
        
        print(f"\n2022 (New Regulations):  {reg_avg:.3f}")
        print(f"2023-2025 (Stable):       {stable_avg:.3f}")
        print(f"Difference:               {diff:+.3f}")
        
        if diff > 0.1:
            print("\n→ Testing is MORE predictive during regulation changes!")
            print("  This is good for 2026 (also a regulation change)")
        elif diff < -0.1:
            print("\n→ Testing is LESS predictive during regulation changes")
        else:
            print("\n→ Similar predictive power")
    
    print("\n" + "="*60)
    print("INTERPRETATION")
    print("="*60)
    print("  >0.7  = Strong correlation (testing is reliable)")
    print("  0.4-0.7 = Moderate correlation (somewhat useful)")
    print("  <0.4  = Weak correlation (limited value)")


SUMMARY

Overall average correlation: 0.343

By year:
  2022 [NEW REGS]  0.137
  2023 [Stable]    0.265
  2024 [Stable]    0.558
  2025 [Stable]    0.482

REGULATION CHANGE COMPARISON

2022 (New Regulations):  0.137
2023-2025 (Stable):       0.422
Difference:               -0.285

→ Testing is LESS predictive during regulation changes

INTERPRETATION
  >0.7  = Strong correlation (testing is reliable)
  0.4-0.7 = Moderate correlation (somewhat useful)
  <0.4  = Weak correlation (limited value)


## Step 6: Visualizations

In [13]:
if len(results) > 0:
    df = pd.DataFrame(results)
    
    # Box plot by year
    fig = go.Figure()
    
    for year in sorted(df['year'].unique()):
        year_data = df[df['year'] == year]
        is_reg = year_data['regulation_change'].iloc[0]
        color = 'red' if is_reg else 'blue'
        name = f"{year} {'(New Regs)' if is_reg else '(Stable)'}"
        
        fig.add_trace(go.Box(
            y=year_data['correlation'],
            name=name,
            marker_color=color,
            boxmean='sd'
        ))
    
    fig.add_hline(y=0.7, line_dash="dash", line_color="green", 
                  annotation_text="Strong (0.7)")
    fig.add_hline(y=0.4, line_dash="dash", line_color="orange", 
                  annotation_text="Moderate (0.4)")
    
    fig.update_layout(
        title='Testing Prediction Accuracy by Year',
        yaxis_title='Spearman Correlation',
        height=500,
        showlegend=True
    )
    fig.show()
    
    # Bar chart by race
    fig2 = px.bar(
        df,
        x='race',
        y='correlation',
        color='regulation_change',
        color_discrete_map={True: 'red', False: 'blue'},
        title='Prediction Accuracy by Race (Red = Regulation Change)',
        labels={'correlation': 'Spearman Correlation', 'race': 'Grand Prix',
                'regulation_change': 'New Regulations'}
    )
    fig2.update_layout(height=500, xaxis_tickangle=-45)
    fig2.show()
else:
    print("No data to visualize")

## Conclusion

**What to look for:**

1. **Is testing predictive?**
   - If correlation >0.7: Testing is a strong predictor
   - If correlation 0.4-0.7: Testing is moderately useful
   - If correlation <0.4: Testing has limited value

2. **Is 2022 different?**
   - If 2022 shows higher correlation: Testing is MORE reliable during regulation changes
   - This would indicate 2026 testing will be reliable (also regulation change)

3. **Application to 2026:**
   - If results are positive: Apply same methodology after 2026 testing
   - Use testing directionality + track characteristics for early season predictions
   - Let learning system adjust as season progresses